In [1]:
!pip install kornia

     |████████████████████████████████| 286kB 5.0MB/s 


In [ ]:
!pip install torchlars

  ERROR: Failed building wheel for torchlars
  Running setup.py clean for torchlars
Failed to build torchlars
    Running setup.py install for torchlars ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-2_r991_p/torchlars/setup.py'"'"'; __file__='"'"'/tmp/pip-install-2_r991_p/torchlars/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-jkva1rvz/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [2]:
import numpy as np
import copy
import time
import random
from functools import wraps
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torchvision.models import squeezenet
from kornia import augmentation as augs
from kornia import filters

from sklearn.model_selection import train_test_split

In [ ]:
train_data=CIFAR10(root="/content/images", train=True, download=True, transform=transforms.ToTensor())
test_data=CIFAR10(root="/content/images", train=False, download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def sort_indices(data, nclass = 2):
  full_labels = np.array(data.targets)
  labels = [i for i in range(10)]
  indices = []
  for label in labels:
    indice = (full_labels == label).nonzero()[0]
    indices.append(indice)

  total_indices=np.array([], dtype=np.int16)
  for i in range(nclass):
    #print(indices[i])
    total_indices = np.append(total_indices, indices[i]) 
  print(total_indices.shape)
  return total_indices

#test_in = sort_indices(test_data, 2)
train_indices = sort_indices(train_data, 2)
test_indices = sort_indices(test_data, 2)

(10000,)
(2000,)


In [ ]:
val_indices = np.append(train_indices[:500], train_indices[-500:])
train_indices = train_indices[500:-500]
train_indices.shape, val_indices.shape

((9000,), (1000,))

In [ ]:
ratio = 0.6
train1_indices = np.append(train_indices[:int(4500*ratio)], train_indices[4500:int(4500*(2-ratio))])
train2_indices = np.append(train_indices[int(4500*(2-ratio)):], train_indices[int(4500*ratio):4500])
train1_indices.shape, train2_indices.shape

((4500,), (4500,))

In [ ]:
def assert_distribution(train_data, train1_indices, train2_indices):
  c1, c2 = 0, 0
  for ix in train1_indices:
    if train_data.targets[ix] == 0:
      c1 += 1
    elif train_data.targets[ix] == 1:
      c2 += 1
  print("Worker 1 Set:-\n")
  print("Label 1 samples = ", c1, " Label 2 samples = " , c2)
  c1, c2 = 0, 0
  for ix in train2_indices:
    if train_data.targets[ix] == 0:
      c1 += 1
    elif train_data.targets[ix] == 1:
      c2 += 1
  print("Worker 1 Set:-\n")
  print("Label 1 samples = ", c1, " Label 2 samples = " , c2)

2700 1800


In [ ]:
batch_size = 300
import random
def create_dataloader(train_data, test_data, train1_indices, train2_indices, val_indices, batch_size):
  batched_indices = []
  no_of_batches = int(len(train1_indices)/batch_size)
  for i in range(no_of_batches):
    batched_indices.append([j for j in range(batch_size*i, batch_size*(i+1))])
  random.shuffle(batched_indices)
  train1_batched_indices = np.array([], dtype=np.int32)
  train2_batched_indices = np.array([], dtype=np.int32)
  for i in range(no_of_batches):
    train1_batched_indices = np.append(train1_batched_indices, train1_indices[batched_indices[i]])
    train2_batched_indices = np.append(train2_batched_indices, train2_indices[batched_indices[i]])
  train1_dataloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = False, sampler = train1_batched_indices)
  train2_dataloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = False, sampler = train2_batched_indices)
  val_dataloader = torch.utils.data.DataLoader(train_data, batch_size = 1024, shuffle = False, sampler = val_indices)
  test_dataloader = torch.utils.data.DataLoader(test_data, batch_size = 1024, shuffle = False, sampler = test_indices)
  return train1_dataloader, train2_dataloader, val_dataloader, test_dataloader

In [ ]:
train1, train2, val, test = create_dataloader(train_data, test_data, train1_indices, train2_indices, val_indices, batch_size)

### Implement Augmentation Function

#### Helper Functions

In [ ]:
# augmentation utils
class RandomApply(nn.Module):
    def __init__(self, fn, p):
        super().__init__()
        self.fn = fn                 # Transform function to apply
        self.p = p                   # Probability of application
    def forward(self, x):
        if random.random() > self.p:
            return x
        return self.fn(x)

In [ ]:
transform_fn = nn.Sequential(RandomApply(augs.ColorJitter(0.8, 0.8, 0.8, 0.2), p=0.8), 
                             augs.RandomGrayscale(p=0.3), 
                             augs.RandomHorizontalFlip(), 
                             RandomApply(filters.GaussianBlur2d((3, 3), (1.5, 1.5)), p=0.2), 
                             augs.RandomResizedCrop((32, 32)))

### Implement Contrastive Loss function

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# queries -> Activation of image, keys -> Activation of Augmented Images
def contrastive_loss(queries, keys, temperature = 0.1):
    b, device = queries.shape[0], queries.device
    logits = queries @ keys.t()
    logits = logits - logits.max(dim=-1, keepdim=True).values
    logits /= temperature
    return F.cross_entropy(logits, torch.arange(b, device=device))


In [ ]:
def softXEnt (input, target):
    logprobs = torch.nn.functional.log_softmax (input, dim = 1)
    return  -(target * logprobs).sum() / input.shape[0]

def Contrastive_Loss(x, y, temperature=750.0):
  batch_size = x.size()[0]
  x = nn.functional.normalize(x, dim=1)
  y = nn.functional.normalize(y, dim=1)
  LARGE_NUM = 1e9
  mask = torch.eye(batch_size)
  label = nn.functional.one_hot(torch.arange(0, batch_size), num_classes = 2*batch_size).type(torch.float32)
  if torch.cuda.is_available():
      mask=mask.cuda()
      label=label.cuda()
  logits_aa = torch.matmul(x, x.t())/temperature
  logits_bb = torch.matmul(y, y.t())/temperature
  logits_ab = torch.matmul(x, y.t())/temperature
  logits_ba = torch.matmul(y, x.t())/temperature

  logits_aa = logits_aa - mask*LARGE_NUM
  logits_bb = logits_bb - mask*LARGE_NUM

  loss_a = softXEnt(torch.cat((logits_ab, logits_aa), dim=1), label)
  loss_b = softXEnt(torch.cat((logits_ba, logits_bb), dim=1), label)

  loss = loss_a + loss_b
  return loss

In [ ]:
class BicontrastiveLoss(nn.Module):
  def __init__(self, margin=1.0, temperature = 1.0, lower_bound = 1e-3):
    super(BicontrastiveLoss, self).__init__()
    self.margin = margin
    self.temperature = temperature
    self.clip_low = lower_bound

  def forward(self, X, Y):
    batch_size, hidden_dim = X.shape
    x = nn.functional.normalize(X, p=2, dim=1)
    y = nn.functional.normalize(Y, p=2, dim=1)
    mask = torch.ones((4*batch_size, batch_size))*self.margin
    if torch.cuda.is_available():
      mask=mask.cuda()
    pos_x = torch.matmul(x, x.t())/self.temperature
    pos_y = torch.matmul(y, y.t())/self.temperature
    neg_xy = torch.matmul(x, y.t())/self.temperature
    neg_yx = torch.matmul(y, x.t())/self.temperature
    v = torch.cat((pos_x, pos_y, neg_xy, neg_yx), dim=0)
    #exp_v = torch.exp(v)
    #soft_v = exp_v/torch.sum(exp_v)
    soft_v = (v-torch.min(v))/(torch.max(v)-torch.min(v))
    #soft_v = torch.clip(soft_v, min=self.clip_low)
    mask[:2*batch_size, :] = 2*soft_v[:2*batch_size, :]
    loss = torch.sum(-torch.log(mask-soft_v))
    return loss

In [3]:
x = torch.rand((32, 128, 128), dtype=torch.float32, requires_grad=False)
y = torch.rand((32, 128, 128), dtype=torch.float32, requires_grad=False)

In [13]:
target1 = torch.round(torch.rand((32), dtype=torch.float32, requires_grad=False)).type(torch.int32)
target2 = torch.round(torch.rand((32), dtype=torch.float32, requires_grad=False)).type(torch.int32)

In [9]:
W1 = nn.Sequential(nn.Linear(128*128, 512),
                  nn.ReLU(),
                  nn.Linear(512, 64)
)
W2 = nn.Sequential(nn.Linear(128*128, 512),
                  nn.ReLU(),
                  nn.Linear(512, 64)
)


In [11]:
X = W1(x.view(x.size(0), -1))
Y = W1(y.view(y.size(0), -1))

In [31]:
def distance_loss(X, Y, target1, target2):
  s = 0
  for ix, t1 in zip(X, target1):
    for iy, t2 in zip(Y, target2):
      if t1 == t2:
        continue
      s += torch.square(ix - iy)
  loss = 1/torch.sum(s)
  return loss

In [27]:
for n, p in W1.named_parameters():
  print(n, p.grad)

0.weight None
0.bias None
2.weight None
2.bias None


In [29]:
loss.backward()

In [ ]:
for n, p in W1.named_parameters():
  print(n, p.grad)

In [32]:
from google.colab import drive
drive.mount("/content/gdrive")  

Mounted at /content/gdrive


In [33]:
cd /content/gdrive/Shareddrives/Disk1/InfoPro-Pytorch/Experiments on CIFAR-SVHN-STL10

/content/gdrive/Shareddrives/Disk1/InfoPro-Pytorch/Experiments on CIFAR-SVHN-STL10


In [46]:
%%shell
CUDA_VISIBLE_DEVICES=0 python train_data.py --dataset cifar10 --model resnet --layers 32 --droprate 0.0 --cos_lr --local_module_num 2  --local_loss_mode cross_entropy --aux_net_widen 1 --aux_net_feature_dim 128 --alpha_1 5 --beta_1 1.0 --beta_2 1.0 --ixx_2 0 --ixy_2 0 --gamma_1 1.0 --gamma_2 0.0 --aux_net_config 1c2f 

Storing in ... ./InfoPro_cifar10_resnet32_K_2_/tuning_weights/no_1_aux_net_config_1c2f_local_loss_mode_cross_entropy_aux_net_widen_1.0_aux_net_feature_dim_128_alpha_1_5.0_beta_1_1.0_beta_2_1.0_ixx_2_0.0_ixy_2_0.0_cos_lr__gamma_1_1.0_gamma_2_0.0_single_backpass
Files already downloaded and verified
Files already downloaded and verified
No of samples used (10000,) and unused (0,)
No of samples used (2000,) and unused (0,)
Worker 1 Set:-
Label 1 samples =  2250  Label 2 samples =  2250
Worker 2 Set:-
Label 1 samples =  2250  Label 2 samples =  2250
W1 Epoch: [0][1/9]	Time 32.461 (32.461)	Loss 0.7680 (0.7680)	Prec@1 34.961 (34.961)	
W2 Epoch: [0][1/9]	Time 32.461 (32.461)	Loss 0.7122 (0.7122)	Prec@1 46.680 (46.680)	
W1 Epoch: [0][2/9]	Time 32.617 (32.539)	Loss 0.7756 (0.7718)	Prec@1 31.445 (33.203)	
W2 Epoch: [0][2/9]	Time 32.617 (32.539)	Loss 0.7417 (0.7269)	Prec@1 40.430 (43.555)	
W1 Epoch: [0][3/9]	Time 31.361 (32.146)	Loss 0.7352 (0.7596)	Prec@1 43.359 (36.589)	
W2 Epoch: [0][3/9]	Time

CalledProcessError: ignored